In [87]:
# %pip install pyinform -Uq

%reload_ext watermark
%watermark

import sys
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing, model_selection, metrics

from tqdm import tqdm

import pyinform

from python.acnets.datasets import load_connectivity

Last updated: 2022-02-21T13:29:07.716468+01:00

Python implementation: CPython
Python version       : 3.9.10
IPython version      : 8.0.1

Compiler    : Clang 11.1.0 
OS          : Darwin
Release     : 21.3.0
Machine     : x86_64
Processor   : i386
CPU cores   : 12
Architecture: 64bit



In [88]:
DATASETS = dict.fromkeys([
  'dosenbach2007',
  'dosenbach2010',
  # 'difumo_64_2',
  # 'difumo_128_2',
  # 'difumo_1024_2'
  ], None)

# load the datasets
DATASETS = {
  ds_name: xr.open_dataset(f'data/julia2018_resting/timeseries_{ds_name}.nc')
  for ds_name in DATASETS.keys()}


In [91]:
%%script echo skipping...
# DEBUG /start
# EXAMPLE:

xx = np.array([
  [0,1,2,1,1,0,0,0,0],
  [150,300,150,150,0,0,0,0,0],
  [-10,5,-10,-10,0,0,0,0,0],
])

# normalize
xx = (xx - xx.mean(axis=1, keepdims=True)) / (xx.std(axis=1, keepdims=True) + sys.float_info.epsilon)

# binarize
xx, *_ = pyinform.utils.bin_series(xx, b=101)

entropies = np.zeros((xx.shape[0], xx.shape[0]))

for i, source in enumerate(xx):
  for j, target in enumerate(xx):
    te = pyinform.transfer_entropy(source, target, k=1)
    entropies[i,j] = te
    print(source, target, te)

entropies

# DEBUG /end

skipping...


In [120]:
%reload_ext autoreload
%autoreload 3

X = [X_subj.values
     for X_subj
     in DATASETS['dosenbach2007']['timeseries'].transpose('subject', 'timestep', 'region')]

from python.acnets.connectome import ExtraConnectivityMeasure

X_te = ExtraConnectivityMeasure(kind='transfer_entropy').fit_transform(X)

100%|██████████| 32/32 [00:06<00:00,  4.67it/s]
